In [4]:
import h2o
import pandas as pd
import numpy as np

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,6 mins 12 secs
H2O_cluster_timezone:,Asia/Kolkata
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.4
H2O_cluster_version_age:,19 days
H2O_cluster_name:,H2O_from_python_Raunak_a4jd7o
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2.973 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [3]:
data = h2o.import_file("http://s3.amazonaws.com/h2o-public-test-data/smalldata/airlines/allyears2k_headers.zip")

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train, valid, test = data.split_frame([0.8,0.1], seed = 42)

In [8]:
print("%d/%d/%d" % (train.nrows, valid.nrows, test.nrows))

35258/4268/4452


In [11]:
train2 = data[1:35255,:]
train2 = h2o.assign(train2, 'first35255')

In [12]:
h2o.ls()

,key
0,Key_Frame__http___s3_amazonaws_com_h2o_public_...
1,first35255
2,py_2_sid_bc6a
3,py_3_sid_bc6a
4,py_4_sid_bc6a
5,py_5_sid_bc6a


In [13]:
data.ncol

31

In [14]:
dates = data[:,0:4]

In [16]:
airports = data[:,['Origin', 'Dest']]

In [17]:
dates.ncol

4

In [18]:
airports.ncol

2

### Use cbind to join columns

In [19]:
a_and_d = airports.cbind(dates)

In [20]:
a_and_d.dim

[43978, 6]

### Use rbind to join rows

In [21]:
restored_data = train.rbind([valid, test])

In [22]:
restored_data.dim

[43978, 31]

In [23]:
data.dim

[43978, 31]

In [24]:
restored_data[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


In [25]:
data[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


In [26]:
train[:,0:4].head()

Year,Month,DayofMonth,DayOfWeek
1987,10,14,3
1987,10,15,4
1987,10,17,6
1987,10,18,7
1987,10,19,1
1987,10,21,3
1987,10,22,4
1987,10,23,5
1987,10,24,6
1987,10,25,7


### Merging

###### Datasets can have different number of rows and columns. It works like join in SQL

In [27]:
A = h2o.H2OFrame(
    pd.DataFrame({
        'a':range(1,11),
        'b':range(11,21),
        'c':range(21,31)
    }),
    destination_frame="A"
    )

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [29]:
A.as_data_frame()

,a,b,c
0,1,11,21
1,2,12,22
2,3,13,23
3,4,14,24
4,5,15,25
5,6,16,26
6,7,17,27
7,8,18,28
8,9,19,29
9,10,20,30


In [30]:
np.random.seed(123)
B = h2o.H2OFrame(
    pd.DataFrame({
        'a':np.random.randint(6,13,20),
        'd':np.random.randint(6,13,20)
    }),
    destination_frame="B"
    )

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [31]:
B.as_data_frame()

,a,d
0,12,6
1,11,7
2,12,9
3,8,12
4,10,11
5,8,10
6,12,6
7,7,6
8,9,10
9,8,7


In [32]:
M = A.merge(B)

In [33]:
M

a,b,c,d
6,16,26,10
6,16,26,7
7,17,27,6
7,17,27,8
7,17,27,8
7,17,27,6
7,17,27,6
8,18,28,12
8,18,28,10
8,18,28,7
